<a href="https://colab.research.google.com/github/peisuke/generative_ai_notebooks/blob/main/28_SeamlessM4T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# パッケージのインストール
!git clone https://github.com/facebookresearch/seamless_communication
%cd seamless_communication

Cloning into 'seamless_communication'...
remote: Enumerating objects: 4247, done.
remote: Counting objects: 100% (785/785), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 4247 (delta 664), reused 637 (delta 590), pack-reused 3462
Receiving objects: 100% (4247/4247), 6.32 MiB | 18.12 MiB/s, done.
Resolving deltas: 100% (2685/2685), done.
/content/seamless_communication


In [3]:
!git reset --hard 65617b6

HEAD is now at 65617b6 Merge pull request #112 from facebookresearch/header_fix


In [4]:
!pip install .

Processing /content/seamless_communication
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.3/162.3 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [5]:
import torch
from seamless_communication.models.inference import Translator

# トランスレーターの準備
translator = Translator(
    "seamlessM4T_medium",
    vocoder_name_or_card="vocoder_36langs",
    dtype=torch.float16,
    device=torch.device("cuda:0")
)

100%|██████████| 6.37G/6.37G [02:27<00:00, 46.3MB/s]
100%|██████████| 4.63M/4.63M [00:00<00:00, 49.3MB/s]
100%|██████████| 160M/160M [00:00<00:00, 185MB/s]


In [6]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
import torchaudio

path_to_input_audio="/content/drive/MyDrive/AI/VALL-E-X/my_voice_sample.wav"
path_to_resampled_audio="sample_160000.wav"

# 16000Hzにリサンプリング
resample_rate = 16000
waveform, sample_rate = torchaudio.load(path_to_input_audio)
resampler = torchaudio.transforms.Resample(sample_rate, resample_rate, dtype=waveform.dtype)
resampled_waveform = resampler(waveform)
torchaudio.save(path_to_resampled_audio, resampled_waveform, resample_rate)

In [9]:
# S2ST　(音声から音声への翻訳)
translated_text, wav, sr = translator.predict("sample_160000.wav", "s2st", "eng")
print("translated_text:", translated_text)
print("wav", wav)
print("sr:", sr)

translated_text: I'm sorry about that. I'm sorry about that.
wav tensor([[[-0.0002, -0.0003, -0.0001,  ..., -0.0525, -0.0456, -0.0498]]],
       device='cuda:0')
sr: 16000


In [10]:
# WAVファイルの保存
torchaudio.save("sample_en.wav", wav[0].cpu(), sr)

In [11]:
from IPython.display import Audio
Audio("sample_en.wav", rate=16000)